In [1]:
import pandas as pd
import numpy as np
from surprise import CoClustering
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise.model_selection import KFold

from read_and_split_data import split_data, filter_animes_without_grade

In [2]:
def get_dataset():
    anime = pd.read_parquet("../datasets/anime.parquet")
    anime = anime[["anime_id", "type"]]
    users = pd.read_parquet("../datasets/users.parquet")
    base_df = users.merge(anime, on="anime_id", how="left")
    return base_df

base_df = get_dataset()
data = split_data(base_df)

In [3]:
kf = KFold(n_splits=5)

In [4]:
def predict_and_error(data, algo, path):
    fold = 0
    predictions_list = []
    for trainset, testset in kf.split(data):
        algo.fit(trainset)
        predictions = algo.test(testset)
        predictions_list.append(predictions)
        # Mean Squared Error
        accuracy.rmse(predictions, verbose=True)
        # Mean Absolute Erro
        accuracy.mse(predictions, verbose=True)
        # Mean Absolute Erro
        accuracy.mae(predictions, verbose=True)
        # Fraction of Concordant Pairs
        accuracy.fcp(predictions, verbose=True)
        base_df = pd.DataFrame(columns=["user", "anime", "actual", "est", "details"], data=predictions)
        base_df.to_parquet(f"{path}_{fold}.parquet")
        fold += 1
    return predictions

In [5]:
algo_normal_predictor = CoClustering()

In [6]:
predictions = predict_and_error(data, algo_normal_predictor, path="../predictions/co_clustering/anime_type_tv/no_filter/co_clustering")

RMSE: 2.1808
MSE: 4.7558
MAE:  1.5027
FCP:  0.6855
RMSE: 2.1728
MSE: 4.7213
MAE:  1.4954
FCP:  0.6888
RMSE: 2.1672
MSE: 4.6966
MAE:  1.4983
FCP:  0.6882
RMSE: 2.1791
MSE: 4.7483
MAE:  1.5028
FCP:  0.6832
RMSE: 2.1672
MSE: 4.6969
MAE:  1.4936
FCP:  0.6905


In [7]:
base_df_without_negative = filter_animes_without_grade(base_df)

In [8]:
predictions_without_negative = predict_and_error(data, algo_normal_predictor, path="../predictions/co_clustering/anime_type_tv/with_filter_remove_negative/co_clustering")

RMSE: 2.1643
MSE: 4.6844
MAE:  1.4955
FCP:  0.6905
RMSE: 2.1661
MSE: 4.6921
MAE:  1.4925
FCP:  0.6897
RMSE: 2.1732
MSE: 4.7226
MAE:  1.4979
FCP:  0.6881
RMSE: 2.1745
MSE: 4.7287
MAE:  1.5030
FCP:  0.6898
RMSE: 2.1643
MSE: 4.6842
MAE:  1.4916
FCP:  0.6894
